In [89]:
import time 
from bs4 import BeautifulSoup
import requests
import pandas as pd
import random
import datetime
import re


In [42]:
df = pd.read_csv('daily_update.csv', index_col=0, parse_dates=True, names = ['date', 'vaccine_num_total', 'vaccine_num_daily', 'link'])
df = df.iloc[1:]
df['date'] = pd.to_datetime(df['date'])
df['date'] = df['date'].dt.date
df['vaccine_num_total'] = df['vaccine_num_total'].astype(int)
df['vaccine_num_daily'] = df['vaccine_num_daily'].astype(int)
df.dtypes

date                 object
vaccine_num_total     int64
vaccine_num_daily     int64
link                 object
dtype: object

In [103]:
df.head()

latest_date_org = df['date'].max()
latest_date_org
latest_cavvine_num = df[df['date'] == latest_date_org]['vaccine_num_total'].values[0]

In [83]:
url = 'http://sousuo.gov.cn/column/40403/0.htm'
response = requests.get(url)
response.encoding = 'utf-8'
soup = BeautifulSoup(response.text, 'html.parser')
soup



<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">

<html xmlns="http://www.w3.org/1999/xhtml">
<head>
<script charset="utf-8" id="allmobilize" src="http://ysp.www.gov.cn/efb21e959cfc1f75725c5f5df95ee8ff/allmobilize.min.js"></script>
<meta content="no-siteapp" http-equiv="Cache-Control"/>
<meta content="width=1100" name="viewport"/>
<link href="#" media="handheld" rel="alternate"/>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="IE=8" http-equiv="X-UA-Compatible"/>
<meta content="" name="description"/>
<meta content="" name="keywords"/>
<meta content="noindex,nofollow,noarchive" name="robots"/>
<title>服务信息(1)</title>
<link href="http://sousuo.gov.cn/static/css/base.css" rel="stylesheet" type="text/css"/>
<link href="http://sousuo.gov.cn/static/css/common.css" rel="stylesheet" type="text/css"/>
<link href="http://sousuo.gov.cn/static/css/columnlist.css" rel="stylesheet" type="text

In [85]:
links = soup.find_all('li')
max_date = datetime.date(2020, 12, 1)
max_date_link = ''
for i in links:
    if '新冠病毒疫苗接种情况' in i.text:
        vaccine_announce_date = i.a['href'][25:35]
        vaccine_announce_date = datetime.datetime.strptime(vaccine_announce_date, '%Y-%m/%d').date()
        vaccine_announce_date = vaccine_announce_date + datetime.timedelta(days=-1)

        if vaccine_announce_date > max_date:
            max_date = vaccine_announce_date
            link = i.a['href']
            max_date_link = link

print(max_date)
print(max_date_link)


2022-12-13
http://www.gov.cn/xinwen/2022-12/14/content_5731922.htm


In [100]:
response = requests.get(max_date_link)
response.encoding = 'utf-8'
soup = BeautifulSoup(response.text, 'html.parser')
soup

page_text =  soup.find('p').text
print(page_text)
text_extract = page_text[20:]
text_num = re.findall(r'(\d+\.\d+)', text_extract)
text_num = text_num[0]
vaccine_num = int(float(text_num)*10000)
vaccine_num
text_date = re.findall(r'(\d{4}年\d{1,2}月\d{1,2}日)', page_text)[0]
text_date = datetime.datetime.strptime(text_date, '%Y年%m月%d日').date()
text_date



截至2022年12月13日，31个省（自治区、直辖市）和新疆生产建设兵团累计报告接种新冠病毒疫苗345167.7万剂次。


datetime.date(2022, 12, 13)

In [ ]:
if max_date > latest_date_org:
    url = max_date_link
    response = requests.get(url)
    response.encoding = 'utf-8'
    soup = BeautifulSoup(response.text, 'html.parser')
    soup
    page_text =  soup.find('p').text
    text_extract = page_text[20:]
    text_num = re.findall(r'(\d+\.\d+)', text_extract)
    text_num = text_num[0]
    vaccine_num = int(float(text_num)*10000)
    print(vaccine_num)
    text_date = re.findall(r'(\d{4}年\d{1,2}月\d{1,2}日)', page_text)[0]
    text_date = datetime.datetime.strptime(text_date, '%Y年%m月%d日').date()
    print(text_date)
    vaccine_num_daily = vaccine_num - latest_cavvine_num
    print(vaccine_num_daily)
    print(page_text)
    df = df.append({'date': text_date, 'vaccine_num_total': vaccine_num, 'vaccine_num_daily': vaccine_num_daily, 'link': url}, ignore_index=True)
    df.to_csv('daily_update.csv')
    print('New data added')
else:
    print('No new data')


#make it run every 4 hours
time.sleep(14400)


In [109]:
import time 
from bs4 import BeautifulSoup
import requests
import pandas as pd
import random
import datetime
import re

df = pd.read_csv('daily_update.csv', index_col=0, parse_dates=True, names = ['date', 'vaccine_num_total', 'vaccine_num_daily', 'link'])
df = df.iloc[1:]
df['date'] = pd.to_datetime(df['date'])
df['date'] = df['date'].dt.date
df['vaccine_num_total'] = df['vaccine_num_total'].astype(int)
df['vaccine_num_daily'] = df['vaccine_num_daily'].astype(int)

latest_date_org = df['date'].max()
latest_cavvine_num = df[df['date'] == latest_date_org]['vaccine_num_total'].values[0]

# Get the official link of the latest vaccine data
url = 'http://sousuo.gov.cn/column/40403/0.htm'
response = requests.get(url)
response.encoding = 'utf-8'
soup = BeautifulSoup(response.text, 'html.parser')

links = soup.find_all('li')
max_date = datetime.date(2020, 12, 1)
max_date_link = ''
for i in links:
    if '新冠病毒疫苗接种情况' in i.text:
        vaccine_announce_date = i.a['href'][25:35]
        vaccine_announce_date = datetime.datetime.strptime(vaccine_announce_date, '%Y-%m/%d').date()
        vaccine_announce_date = vaccine_announce_date + datetime.timedelta(days=-1)

        if vaccine_announce_date > max_date:
            max_date = vaccine_announce_date
            link = i.a['href']
            max_date_link = link
print(max_date)
print(max_date_link)


# Get the latest vaccine data
if max_date > latest_date_org:
    url = max_date_link
    response = requests.get(url)
    response.encoding = 'utf-8'
    soup = BeautifulSoup(response.text, 'html.parser')
    soup
    page_text =  soup.find('p').text
    text_extract = page_text[20:]
    text_num = re.findall(r'(\d+\.\d+)', text_extract)
    text_num = text_num[0]
    vaccine_num = int(float(text_num)*10000)
    print(vaccine_num)
    text_date = re.findall(r'(\d{4}年\d{1,2}月\d{1,2}日)', page_text)[0]
    text_date = datetime.datetime.strptime(text_date, '%Y年%m月%d日').date()
    print(text_date)
    vaccine_num_daily = vaccine_num - latest_cavvine_num
    print(vaccine_num_daily)
    print(page_text)
    df = df.append({'date': text_date, 'vaccine_num_total': vaccine_num, 'vaccine_num_daily': vaccine_num_daily, 'link': url}, ignore_index=True)
    df.to_csv('daily_update.csv')
    print('New data added')
else:
    print('No new data')

#make it run every 4 hours
time.sleep(14400)


2022-12-13
http://www.gov.cn/xinwen/2022-12/14/content_5731922.htm
No new data


KeyboardInterrupt: 